In [113]:
import pandas as pd
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [114]:
import csv

with open("/content/spotify_millsongdata.csv", 'r', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)





In [115]:
df.head(5)

,artist,song,text
0,Barbra Streisand,Everything Must Change,everything must change nothing stays the same ...
1,Counting Crows,Ooh-La-La,"poor old granddad , i laughed at all his words..."
2,George Strait,Is It Already Time,the years have been so good to you and i my fr...
3,Fleetwood Mac,Homework,"oh baby , you know i may be a fool i 'm wastin..."
4,Christina Aguilera,Hello,"oh ooh , yeah does it ever feel like no one is..."


In [116]:
df.tail(5)

,artist,song,text
4995,Face To Face,Fight Or Flight,we 're ready to go ticking like a bomb that 's...
4996,Barbra Streisand,I Remember,i awake on a chilly christmas morning watching...
4997,Gipsy Kings,A Tu Vera,to your side to your side always the verita yo...
4998,Free,Walk In My Shadow,walk in my shadow my throat is dry my knees ar...
4999,Green Day,Geek Stink Breath,i 'm on a mission i made my decision to lead a...


In [117]:
df.shape

(5000, 3)

In [118]:
df.isnull().sum()

artist    0
song      0
text      0
dtype: int64

In [119]:

if 'link' in df.columns:
    # Drop the 'link' column
    df = df.sample(5000).drop('link', axis=1).reset_index(drop=True)
else:
    # Handle the case where the 'link' column doesn't exist
    print("The 'link' column does not exist in the DataFrame.")

# Continue with your data processing as needed


The 'link' column does not exist in the DataFrame.


In [120]:
df.head(10)

,artist,song,text
0,Barbra Streisand,Everything Must Change,everything must change nothing stays the same ...
1,Counting Crows,Ooh-La-La,"poor old granddad , i laughed at all his words..."
2,George Strait,Is It Already Time,the years have been so good to you and i my fr...
3,Fleetwood Mac,Homework,"oh baby , you know i may be a fool i 'm wastin..."
4,Christina Aguilera,Hello,"oh ooh , yeah does it ever feel like no one is..."
5,Chris Rea,Daytona,diamond dream when the sun do n't shine i clos...
6,David Bowie,Future Legend,"and in the death , as the last few corpses lay..."
7,Frank Sinatra,A Lover Is Blue,"yesterday we were gay , i was high on a rainbo..."
8,ABBA,Hamlet III,when the autumn leaves are falling to the grou...
9,Arlo Guthrie,Miss The Mississippi And You,i 'm growing tired of the big city lights tire...


In [121]:
df['text'][0]

"everything must change nothing stays the same everyone must change no one stays the same the young become the old and mysteries do unfold cause that 's the way of time nothing and no one goes unchanged there are not many things in life you can be sure of except rain comes from the clouds sun lights up the sky and hummingbirds do fly winter turns to spring a wounded heart will heal but never much too soon yes everything must change the young become the old and mysteries do unfold cause that 's the way of time nothing and no one goes unchanged there are not many things in life you can be sure of except rain comes from the clouds sun lights up the sky and butterflies do fly rain comes from the clouds sun lights up the sky and music and music makes me cry"

In [122]:
# df = df.sample(5000)

In [123]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [124]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [125]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [126]:


# Define a tokenization function
def tokenization(text):
    try:
        tokens = nltk.word_tokenize(text)
    except (TypeError, ValueError) as e:
        print(f"Tokenization error for text: {text}. Error: {e}")
        tokens = []  # Handle the error gracefully
    return tokens

# Apply tokenization to the 'text' column
df['text'] = df['text'].apply(lambda x: tokenization(x))


In [127]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Convert the lists of tokens back to a single string for each row
df['text'] = df['text'].apply(lambda tokens: ' '.join(tokens))

# Create the TF-IDF matrix
tfidf_vectorizer = TfidfVectorizer(analyzer='word', stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df['text'])

# Calculate cosine similarity
similarity = cosine_similarity(tfidf_matrix)


In [128]:
similarity[0]

array([1.        , 0.00920161, 0.03188576, ..., 0.        , 0.00938927,
       0.00233246])

In [129]:
df[df['song'] == 'Crying Over You']

,artist,song,text


In [130]:
def recommendation(song_df):
    if song_df not in df['song'].values:
        return f"Song '{song_df}' not found in the dataset."

    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])), reverse=True, key=lambda x: x[1])

    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)

    return songs

# Call the recommendation function with 'Crying Over You' or any other song title
result = recommendation('Crying Over You')

if isinstance(result, str):
    print(result)
else:
    print("Recommended songs:", result)


Song 'Crying Over You' not found in the dataset.


In [131]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))